In [1]:
import os
import sys
import random
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold, GroupShuffleSplit, GridSearchCV
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.utils.validation import check_array, check_random_state, _deprecate_positional_args
from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits
from scipy.interpolate import make_interp_spline
from functions import *

In [ ]:
# set device to GPU if available

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")

In [ ]:
# cycle vs capacity retention

file_path = "capacity list.xlsx"
data = pd.read_excel(file_path)
cycle_numbers = data.columns[2:].astype(int)

last_cycle_values = data.iloc[:, -1].values # set colors based on final capacity
norm = plt.Normalize(vmin=last_cycle_values.min(), vmax=last_cycle_values.max())
cmap = plt.get_cmap('viridis').reversed()

fig, ax = plt.subplots(figsize=(8, 5))
y_min, y_max = 2.75, 3.25

for index, row in data.iterrows():
    y = row[2:].values  
    x_new = np.linspace(cycle_numbers.min(), cycle_numbers.max(), 300)
    spline = make_interp_spline(cycle_numbers, y, k=3)
    y_smooth = spline(x_new)    
    cell_color = cmap(norm(row.iloc[-2]))
    ax.plot(x_new, y_smooth, color=cell_color, linewidth=1.5)
    ax.scatter(cycle_numbers, y, color=cell_color, s=25, marker='o', zorder=3)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = fig.colorbar(sm, ax=ax, fraction=0.03, pad=0.04)
cbar.set_label('Capacity (Ah)')
ax.set_ylim([y_min, y_max])
ax.set_xlabel('Cycle Number')
ax.set_ylabel('Capacity (Ah)')
ax.set_title('Cycle Number vs Capacity for Each Cell')
ax.grid(True)

plt.show()

In [ ]:
# Model performance with more dynamic operation data

# input data
diagcap_final = current_health_state('diagnosis', 'capacity list.xlsx')
sequence = torch.tensor(pd.read_excel('sequence list_index.xlsx', header=None).values, dtype=torch.long)

# Select mode
mode = 'sequence-only' # 'full', 'diagnosis-only', 'sequence-only'

if mode == 'diagnosis-only':
    sequence = torch.zeros_like(sequence)
elif mode == 'sequence-only':
    diagcap_final = torch.zeros_like(diagcap_final)

# target data
futureVR = process_folder_futureDI('diagnosis')

# Hyperparameters
input_dim_encoder = 11
n_cat = 4
hidden_dim = 64
output_dim = 10
randomst = 0
num_folds = 6
batch_size = 6
num_encoder_fc_layers = 2
num_gru_layers = 2
num_decoder_fc_layers = 0  
date = 241009


# Results folder 
result_folder = f'degradation path prediction_{mode}_{date}'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)
        
# perform prediction
prediction(diagcap_final, sequence, futureVR, input_dim_encoder, hidden_dim, output_dim,
           n_cat, num_encoder_fc_layers, num_gru_layers, num_decoder_fc_layers, device, 
           batch_size, mode, result_folder, model_path='Best model.pt')

In [ ]:
# future capacity trajectory prediction

capacity = torch.tensor(pd.read_excel('capacity list.xlsx').iloc[:, 3:].values, dtype=torch.float32)

capacity_folder = 'capacity_per_fold'
if not os.path.exists(capacity_folder):
    os.makedirs(capacity_folder)
    
# Hyperparameters
randomst = 0
num_folds = 4

# future capacity for various forecast length and folds
target_capacity(capacity, num_folds, randomst, capacity_folder)

#Unified regression model with true DI and capacity
result_folder = f'degradation path prediction_{mode}_{date}'
ref_true = 1
n_true = 1
n_folds = 4
model = unified_regression(result_folder, capacity_folder, n_folds, n_true, ref_true)

#Prediction using the unified regression model with predicted DI as input
evaluate_predictions(result_folder, capacity_folder, model)